In [1]:
import os
import sys; sys.path.append("~/HHbbgg_ETH_devel/Training/python") # to load packages
import training_utils as utils
import numpy as np
reload(utils)
import preprocessing_utils as preprocessing
reload(preprocessing)
import plotting_utils as plotting
reload(plotting)
import optimization_utils as optimization
reload(optimization)
import postprocessing_utils as postprocessing
reload(postprocessing)
import pandas as pd
import root_pandas as rpd
import matplotlib.pyplot as plt


Welcome to ROOTaaS 6.06/02


In [2]:
#dirpath='/work/nchernya/CMSSW_9_4_9/src/flashgg/output_trees_nottH_properSM2016GEN_11_03_2019/output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph.root'
dirpath='/work/nchernya/CMSSW_9_4_9/src/flashgg/output_trees_nottH_properSM2017_11_03_2019/output_GluGluToHHTo2B2G_node_SM_13TeV-madgraph.root'

treename='GluGluToHHTo2B2G_node_SM_13TeV_madgraph_13TeV_DoubleHTag_0'
treepath='tagsDumper/trees/'

In [3]:
df = rpd.read_root(dirpath,treepath+treename)

In [4]:
print any(df.isnull().any()) # no NANs in the dataframe

False


In [5]:
import ROOT
from ROOT import TFile, TH1F, TCanvas, gROOT, TLorentzVector,TH1,TH1F

In [6]:
final_dijetM = []
final_dijetPt = []
final_diHiggsM=[]
final_diHiggsPt=[]
final_MX=[]
final_jet1_pt=[]
final_jet2_pt=[]
for index, row in df.iterrows():
    dijetM = 0
    dijetPt = 0
    diHiggsM = 0
    diHiggsPt = 0
    jet1, jet2, diphoton = TLorentzVector(),TLorentzVector(),TLorentzVector()
    jet1_reg, jet2_reg = TLorentzVector(),TLorentzVector()
    jet1.SetPtEtaPhiM(row['leadingJet_pt']/row['leadingJet_bRegNNCorr'],row['leadingJet_eta'],row['leadingJet_phi'],row['leadingJet_mass'])
    jet2.SetPtEtaPhiM(row['subleadingJet_pt']/row['subleadingJet_bRegNNCorr'],row['subleadingJet_eta'],row['subleadingJet_phi'],row['subleadingJet_mass'])
    diphoton.SetPtEtaPhiM(row['diphoton_pt'],row['diphoton_eta'],row['diphoton_phi'],row['CMS_hgg_mass'])
    dijetM = (jet1+jet2).M()
    dijetPt = (jet1+jet2).Pt()
    diHiggsM = (jet1+jet2+diphoton).M()
    diHiggsPt = (jet1+jet2+diphoton).Pt()

    final_dijetM.append(dijetM)
    final_dijetPt.append(dijetPt)     
    final_diHiggsM.append(diHiggsM)
    final_diHiggsPt.append(diHiggsPt)
    final_MX.append(diHiggsM - dijetM-diphoton.M()+250)
    final_jet1_pt.append(jet1.Pt())
    final_jet2_pt.append(jet2.Pt())



In [ ]:
df['dijetM_notregressed'] = final_dijetM
df['dijetPt_notregressed'] = final_dijetPt
df['diHiggsPt_notregressed'] = final_diHiggsPt
df['diHiggsM_notregressed'] = final_diHiggsM
df['MX_notregressed'] = final_MX
df['dijetPt_higgsM_notregressed'] = [final_dijetPt[i]/final_diHiggsM[i] for i in range(len(final_diHiggsM))]
df['dijetPt_higgsM'] = df['dijet_pt']/df['diHiggs_mass']
df['jet1_pt_notregressed'] = final_jet1_pt
df['jet2_pt_notregressed'] = final_jet2_pt

In [ ]:
cut = 'jet1_pt_notregressed>20 & jet2_pt_notregressed>20 & subleadingJet_pt>20 & leadingJet_pt>20 '
df = df.query(cut)

In [ ]:
jets='subleading,leading'.split(',')
ofwhat='pt,eta'.split(',')
ranges = [[20,300],[-3.,3]]
binning = [50,20]
colors=['r','g']
colors2=['b','c']
markers=['s','o']

In [ ]:
pathsave='/mnt/t3nfs01/data01/shome/nchernya/HHbbgg_ETH_devel/Training/plots/applied_breg_ptcut/21_03_2019/'

In [ ]:
for num_what,what in enumerate(ofwhat):
    for num,obj in enumerate(jets):
        X_what = (df['%sJet_%s'%(obj,what)]).values.reshape(-1,1)
        res = (df['%sJet_bRegNNResolution'%(obj)]).values.reshape(-1,1)
        ran = ranges[num_what]
        bins = binning[num_what]
        
        bins, y_mean_pt, y_std_pt, y_qt_pt = preprocessing.profile(res,X_what,range=ran,bins=bins,uniform=False,quantiles=np.array([0.25,0.4,0.5,0.75]))
        y_median_pt = y_qt_pt[2]
        y_25_pt,y_40_pt,y_75_pt = y_qt_pt[0],y_qt_pt[1],y_qt_pt[3]
        y_iqr2_pt =  y_qt_pt[0],y_qt_pt[3]
        err_iqr2 =  0.5*(y_qt_pt[3]-y_qt_pt[0])
        binc = 0.5*(bins[1:]+bins[:-1])
     #   print bins[:10]

       # plt.scatter(binc,y_mean_pt,color=colors[num],marker=markers[num],label='%s Jet'%obj) #mean
        plt.scatter(binc,y_median_pt,color=colors[num],marker=markers[num],label='%s Jet'%obj)

    plt.grid(alpha=0.2,linestyle='--',markevery=2)
    axes = plt.gca()
    ymin, ymax = (plt.gca()).get_ylim()
    xmin, xmax = (plt.gca()).get_xlim()
    lgd = plt.legend(loc="upper right")
    plt.xlabel('$%s$'%what)
    plt.ylabel('Median resolution')
    savename='plot_%sJet_%s'%(obj,what)
    plt.savefig(pathsave+savename+'.pdf')
    plt.show()


In [ ]:
for num_what,what in enumerate(ofwhat):
    for num,obj in enumerate(jets):
        X_what = (df['%sJet_%s'%(obj,what)]).values.reshape(-1,1)
        res = (df['%sJet_bRegNNCorr'%(obj)]).values.reshape(-1,1)
        ran = ranges[num_what]
        bins = binning[num_what]
        
        bins, y_mean_pt, y_std_pt, y_qt_pt = preprocessing.profile(res,X_what,range=ran,bins=bins,uniform=False,quantiles=np.array([0.25,0.4,0.5,0.75]))
        y_median_pt = y_qt_pt[2]
        y_25_pt,y_40_pt,y_75_pt = y_qt_pt[0],y_qt_pt[1],y_qt_pt[3]
        y_iqr2_pt =  y_qt_pt[0],y_qt_pt[3]
        err_iqr2 =  0.5*(y_qt_pt[3]-y_qt_pt[0])
        binc = 0.5*(bins[1:]+bins[:-1])
        print bins[:10]

        plt.scatter(binc,y_median_pt,color=colors[num],marker=markers[num],label='%s Jet'%obj)

    plt.grid(alpha=0.2,linestyle='--',markevery=2)
    axes = plt.gca()
    ymin, ymax = (plt.gca()).get_ylim()
    xmin, xmax = (plt.gca()).get_xlim()
    lgd = plt.legend(loc="upper right")
    plt.xlabel('$%s$'%what)
    plt.ylabel('Median correction')
    savename='plot_correction_%sJet_%s'%(obj,what)
    plt.savefig(pathsave+savename+'.pdf')
    plt.show()


### Dijet mass improvement

In [ ]:
plt.clf()
bins = np.linspace(70, 190, 50)
n,bins,patches = plt.hist(df['dijetM_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df['Mjj'],bins=bins,color='red',alpha=0.5,label='regressed')
plt.xlabel('Mjj [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='Mjj_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
import ROOT
import math
from array import array
from ROOT import std
from ROOT import gROOT
from ROOT import gStyle
from ROOT import TH1F, TGraph, TF1
from ROOT import TCanvas
from ROOT import RooRealVar, RooDataHist, RooFormulaVar, RooVoigtian, RooChebychev, RooArgList, \
                 RooArgSet, RooAddPdf, RooDataSet, RooCategory, RooSimultaneous, \
                 RooBreitWigner, RooCBShape, RooFFTConvPdf, RooBukinPdf, RooBifurGauss, RooGenericPdf, RooChi2Var


In [ ]:
gStyle.SetPadTopMargin(0.06)
gStyle.SetPadRightMargin(0.04)
gStyle.SetPadLeftMargin(0.15)

n_bins=50
c_min = min(min(df['Mjj'].values.reshape(-1,1)),min(df['dijetM_notregressed'].values.reshape(-1,1)))
c_max = max(max(df['Mjj'].values.reshape(-1,1)),max(df['dijetM_notregressed'].values.reshape(-1,1)))
c_min=50.
c_max=200.


Histo_rel_diff = np.histogram(df['dijetM_notregressed'].values.reshape(-1,1),bins=n_bins,range=(c_min,c_max))
Histo_rel_diff_reg = np.histogram(df['Mjj'].values.reshape(-1,1),bins=n_bins,range=(c_min,c_max))

rel_diff = df['dijetM_notregressed'].values.reshape(-1,1)
rel_diff_regressed=df['Mjj'].values.reshape(-1,1)

h_rel_diff = TH1F("hrel_diff", "hrel_diff", n_bins, c_min, c_max)
for i in range(len(rel_diff)):
    h_rel_diff.Fill(rel_diff[i])
h_rel_diff_reg = TH1F("hrel_diff_reg", "hrel_diff_reg", n_bins, c_min, c_max)
for i in range(len(rel_diff_regressed)):
    h_rel_diff_reg.Fill(rel_diff_regressed[i])
h_rel_diff.SetLineColor(ROOT.kBlue)
h_rel_diff.SetMarkerColor(ROOT.kBlue)
h_rel_diff.SetLineWidth(2)
h_rel_diff_reg.SetLineColor(ROOT.kRed)
h_rel_diff_reg.SetMarkerColor(ROOT.kRed)
h_rel_diff_reg.SetLineWidth(2)



AllHistos= [Histo_rel_diff, Histo_rel_diff_reg]
h_max = max([histo[0].max() for histo in AllHistos])*1.2
h_min = min([histo[0].min() for histo in AllHistos])

c = ROOT.TCanvas("c","c",900,900)
c.cd()
frame = TH1F("hframe", "hframe", n_bins, c_min, c_max)
frame.SetStats(0)
frame.GetXaxis().SetTitleOffset(0.91);
frame.GetYaxis().SetTitle("Events")
frame.GetXaxis().SetTitle("M(jj) (GeV)")
frame.GetYaxis().SetLabelSize(0.04)
frame.GetYaxis().SetRangeUser(h_min,h_max)

frame.Draw()

plotFolder='/scratch/snx3000/nchernya/bregression/plots/dijet/'


leg = ROOT.TLegend(0.65,0.75,0.9,0.9)
leg.SetFillStyle(-1)
leg.SetBorderSize(0)
leg.SetTextFont(42)
leg.SetTextSize(0.03)
h_rel_diff.Draw("sameHIST")
h_rel_diff_reg.Draw("sameHIST")
leg.AddEntry(h_rel_diff,"Nominal JEC" ,"P")
leg.AddEntry(h_rel_diff_reg,"Regressed" ,"P")
leg.Draw()


print('Nominal : mean, RMS :',h_rel_diff.GetMean(), h_rel_diff.GetRMS())
print('Regresesd : mean, RMS : ',h_rel_diff_reg.GetMean(), h_rel_diff_reg.GetRMS())



c.Draw()

In [ ]:
plt.clf()
bins = np.linspace(0,1, 50)
n,bins,patches = plt.hist(df['dijetPt_higgsM_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df['dijetPt_higgsM'],bins=bins,color='red',alpha=0.5,label='regressed')
plt.xlabel('dijet $p_T$/diHiggs')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='dijetpt_dihiggsM_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
plt.clf()
bins = np.linspace(200, 1000, 60)
n,bins,patches = plt.hist(df['MX_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df['MX'],bins=bins,color='red',alpha=0.5,label='regressed')
plt.xlabel('MX [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='MX_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
plt.clf()
bins = np.linspace(200, 1000, 60)
n,bins,patches = plt.hist(df['diHiggsM_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df['diHiggs_mass'],bins=bins,color='red',alpha=0.5,label='regressed')
n,bins,patches = plt.hist(df['MX'],bins=bins,color='green',alpha=0.5,label='regressed MX')
plt.xlabel('M(HH) [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='Mhh_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
plt.clf()
bins = np.linspace(20, 200, 60)
#n,bins,patches = plt.hist(df.query('leadingJet_pt>20 & subleadingJet_pt>25 & subleadingJet_bDis>0.6')['jet2_pt_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
#n,bins,patches = plt.hist(df.query('leadingJet_pt>20 & subleadingJet_pt>25 & subleadingJet_bDis>0.6')['subleadingJet_pt'],bins=bins,color='red',alpha=0.5,label='regressed')
n,bins,patches = plt.hist(df.query('leadingJet_pt>20 & subleadingJet_pt>30 ')['jet2_pt_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df.query('leadingJet_pt>20 & subleadingJet_pt>30')['subleadingJet_pt'],bins=bins,color='red',alpha=0.5,label='regressed')
plt.xlabel('subleading jet $p_T$ [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='sublead_Jet_pt_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
plt.clf()
bins = np.linspace(20, 200, 60)
n,bins,patches = plt.hist(df['jet1_pt_notregressed'],bins=bins,color='blue',alpha=0.5,label='nominal JEC')
n,bins,patches = plt.hist(df['leadingJet_pt'],bins=bins,color='red',alpha=0.5,label='regressed')
plt.xlabel('leading jet $p_T$ [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
lgd = plt.legend(loc="upper right")
savename='lead_Jet_pt_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
plt.clf()
bins = np.linspace(0, 2, 60)
n,bins,patches = plt.hist(df['subleadingJet_bRegNNCorr'],bins=bins,color='red',alpha=0.5)
plt.yscale('log') 
plt.xlabel('subleading jet $p_T$ [GeV]')
plt.ylabel('Events')
plt.grid(alpha=0.2,linestyle='--',markevery=2)
axes = plt.gca()
#lgd = plt.legend(loc="upper right")
savename='sublead_corr_jetptbin_plot'
plt.savefig(pathsave+savename+'.pdf')
plt.show()

In [ ]:
for num_what,what in enumerate(ofwhat):
    for num,obj in enumerate(jets):
        X_what = (df.query('jet1_pt_notregressed>20 & jet2_pt_notregressed>20')['%sJet_%s'%(obj,what)]).values.reshape(-1,1)
        res = (df.query('jet1_pt_notregressed>20 & jet2_pt_notregressed>20')['%sJet_bRegNNCorr'%(obj)]).values.reshape(-1,1)
        ran = ranges[num_what]
        bins = binning[num_what]
        
        bins, y_mean_pt, y_std_pt, y_qt_pt = preprocessing.profile(res,X_what,range=ran,bins=bins,uniform=False,quantiles=np.array([0.25,0.4,0.5,0.75]))
        y_median_pt = y_qt_pt[2]
        y_25_pt,y_40_pt,y_75_pt = y_qt_pt[0],y_qt_pt[1],y_qt_pt[3]
        y_iqr2_pt =  y_qt_pt[0],y_qt_pt[3]
        err_iqr2 =  0.5*(y_qt_pt[3]-y_qt_pt[0])
        binc = 0.5*(bins[1:]+bins[:-1])
        print bins[:10]

        plt.scatter(binc,y_median_pt,color=colors[num],marker=markers[num],label='%s Jet'%obj)

    plt.grid(alpha=0.2,linestyle='--',markevery=2)
    axes = plt.gca()
    ymin, ymax = (plt.gca()).get_ylim()
    xmin, xmax = (plt.gca()).get_xlim()
    lgd = plt.legend(loc="upper right")
    plt.xlabel('$%s$'%what)
    plt.ylabel('Median correction')
  #  savename='plot_correction_%sJet_%s'%(obj,what)
 #   plt.savefig(pathsave+savename+'.pdf')
    plt.show()


In [ ]:
df.query('leadingJet_pt>30 & subleadingJet_pt>30').shape

In [ ]:
df.shape

In [ ]:
10626./12074

In [ ]:
df.query('leadingJet_pt>20 & subleadingJet_pt>20 & subleadingJet_bDis>0.6').shape

In [ ]:
df.query('leadingJet_pt>20 & subleadingJet_pt>20 ').shape

In [ ]:
df.query('leadingJet_pt>20 & subleadingJet_pt>20 & leadingJet_bDis>0.6').shape

In [ ]:
df.query('subleadingJet_bDis<0.3')['Mjj'].plot.hist(bins=100,range=[60,200])
plt.show()

In [ ]:
df.query('subleadingJet_bDis>=0.3')['Mjj'].plot.hist(bins=100,range=[60,200])
plt.show()